<a href="https://colab.research.google.com/github/GrantBerg/DS-340W/blob/main/DS340w_final_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required libraries installation

In [ ]:
%pip install muspy
%pip install music21

loading libraries into python instance

In [ ]:
import muspy
import random
import os
import music21
import math
from collections import defaultdict
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Defining functions that will be used in the code

In [ ]:
def extract_chord_labels(filepath):
    stream = music21.converter.parse(filepath)
    chords = stream.chordify()
    labels = []
    for c in chords.flat.getElementsByClass('Chord'):
        label = c.pitchedCommonName  # e.g., "C major triad"
        labels.append(label)
    return labels

# Convert chord labels to consistent token format
def tokenize_chords(labels):
    return [label.replace(" ", "_").upper() for label in labels]

def compute_ngram_probs(sequences, n=2):
    model = defaultdict(lambda: defaultdict(int))
    total = 0
    for seq in sequences:
        for i in range(len(seq)-n):
            context = tuple(seq[i:i+n-1])
            target = seq[i+n-1]
            model[context][target] += 1
            total += 1
    # Normalize to get probabilities
    for context in model:
        total_count = sum(model[context].values())
        for token in model[context]:
            model[context][token] /= total_count
    return model

def compute_perplexity(model, sequence, n=2):
    log_prob = 0
    count = 0
    for i in range(len(sequence)-n):
        context = tuple(sequence[i:i+n-1])
        target = sequence[i+n-1]
        prob = model.get(context, {}).get(target, 1e-6)  # smoothing
        log_prob += math.log2(prob)
        count += 1
    return 2 ** (-log_prob / count) if count > 0 else float('inf')

Load the folder containing midi files to be tested

In [ ]:
# midi_folder = 'content/drive/EMOPIA_1.0/EMOPIA_1.0/midi'
# midi_files = [os.path.join(midi_folder, file) for file in os.listdir(midi_folder) if file.endswith(".mid")]
#nes = muspy.NESMusicDatabase()


FileNotFoundError: [Errno 2] No such file or directory: 'content/drive/EMOPIA_1.0/EMOPIA_1.0/midi'

In [ ]:
import muspy
from pathlib import Path  # Add this import
from music21 import converter
class WikifoniaDataset(muspy.RemoteFolderDataset):
    """Wikifonia dataset."""
    _NAME = "Wikifonia"
    _DESCRIPTION = "A dataset of lead sheets with melody and chords."
    _HOMEPAGE = "http://www.synthzone.com/files/Wikifonia/"
    _sources = {
        "wikifonia": {
            "filename": "Wikifonia.zip",
            "url": "http://www.synthzone.com/files/Wikifonia/Wikifonia.zip",
            "archive": True,
            "size": 35727800,
            "md5": "d26e22562e67eb7d37535e96cc5eebba",
            "sha256": "e7bce509462a73cee175308b6a3cdafa9effd6e8958b3ce03b4edb293cc6b691",
        }
    }
    _extension = "mxl"

    def read(self, filename: str | Path) -> muspy.datasets:  # Now Path is defined
      """Read a .mxl file into a Music object."""
      return muspy.read_musicxml(filename)

In [ ]:
wikifonia = WikifoniaDataset(
    root="wikifonia_dataset/",
    download_and_extract=True,
    verbose=True
)

Skip downloading as the `.muspy.success` file is found.
Skip extracting as the `.muspy.success` file is found.


In [ ]:
# test = read("/content/drive/MyDrive/EMOPIA_1.0/EMOPIA_1.0/midis")
# print(test)

for file in wikifonia.raw_filenames:
  print(str(file))

/content/wikifonia_dataset/Wikifonia/ Alessandro Cicognini and Paul Weston, Sammy Cahn - Autumn in Rome.mxl
/content/wikifonia_dataset/Wikifonia/ Burt Bacharach and Carole Bayer Sager - That's What Friends Are For.mxl
/content/wikifonia_dataset/Wikifonia/ Elton John, Bernie Taupin - Goodbye, Yellow Brick Road.mxl
/content/wikifonia_dataset/Wikifonia/ Enrique Santos Discepolo - Cambalache.mxl
/content/wikifonia_dataset/Wikifonia/ Haydn Wood - Roses de Picardie.mxl
/content/wikifonia_dataset/Wikifonia/ Henry Smith 1978 - Give Thanks.mxl
/content/wikifonia_dataset/Wikifonia/ Jacques Offenbach - The Can Can (The Galop from Orpheus in the Underworld).mxl
/content/wikifonia_dataset/Wikifonia/ John W Wright, John W Wright - Dancing.mxl
/content/wikifonia_dataset/Wikifonia/ Kendall Hayes, Gary Walker, Kendall Hayes - Walk On By.mxl
/content/wikifonia_dataset/Wikifonia/ Luigi Denza, Edward Oxenford - Funiculi,  Funicula.mxl
/content/wikifonia_dataset/Wikifonia/ MGMT - ELECTRIC FEEL.mxl
/content

labels and tokenizes data fo input into model

In [ ]:
chord_sequences = []


for file in wikifonia.raw_filenames:
    if str(file).endswith(".mxl"):
        labels = extract_chord_labels(os.path.join(str(file)))
        tokens = tokenize_chords(labels)
        if len(tokens) > 0:
            chord_sequences.append(tokens)

/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/usr/local/lib/python3.11/dist-packages/music21/musicxml/xmlToM21.py:2206: MusicXMLWarning: Warning: measure 50 in part Melodyis overfull: 215/48 > 4.0,assuming 4.0 is correct.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/music21/musicxml/xmlToM21.py:2206: MusicXMLWarning: Warning: measure 62 in part Melodyis overfull: 241/60 > 4.0,assuming 4.0 is correct.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/music21/musicxml/xmlToM21.py:1961: MusicXMLWarning: The following exception took place in m. 27 in part Trumpet.
  warnings.warn(


PitchException: Cannot make a step out of "'"

test and train the model and calculate perplexity

In [ ]:
test_seq = chord_sequences[0]
train_seqs = chord_sequences[1:]

model = compute_ngram_probs(train_seqs, n=2)
pp = compute_perplexity(model, test_seq, n=2)

print(f"Perplexity of test chord sequence: {pp:.4f}")

Perplexity of test chord sequence: 205.5542
